In [2]:
import open3d
import numpy as np
import util
import matplotlib.pyplot as plt
import math
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


**1. Optimization**: You are given the function:  exp(a * x) * sin(x) + b. Implement Levenberg Marquadt using numpy and solve for the parameters of the above function. Optimize for the following parameters: a=2, b = 1. Do this for 50 observations that lie between 1 and 20. Plot the loss values over time and data fit curves. Ensure that your initial estimates are not very close to the final parameters. Write down the jacobian formula in the notebook. **\[3 points\]**

## Jacobian Formula

$$ 
       J = \left[ -x*e^{-a*x}*sin(x) ;  1 \right]
$$ 

## Function 

In [ ]:
def make_gaussian(x,a,b):
    d = np.exp(-a*x)*np.sin(x)+b
    return d

def da(x,a,b):
    d = -x*np.exp(-a*x)*np.sin(x)
    return d

def db(x,a,b):
    return np.linspace(1, 1, 50)

def plot_ps(x, y, w, cost):
    plt.subplot(121)
    plt.plot(x, y, "-", label="Ground Truth")
    plt.plot(x, make_gaussian(x, *w), "-o", label="Predicted Value")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Actual vs. Predicted")
    plt.legend()
    plt.subplot(122)
    plt.plot(cost, "-o")
    plt.title("Loss vs. Iterations")
    plt.xlabel("number of iterations")
    plt.ylabel("loss")
    plt.show()

## Intialization

In [ ]:
num_obs = 50
a_gt = 2
b_gt = 1
x = np.linspace(1, 20, num_obs)
y = make_gaussian(x, a_gt, b_gt)
w = np.array([100,200]) # a,b 
lr = 0.01
tol = 10 ** -100
num_itre = 200

## Iteration

In [ ]:
cost = []
for iteration in range(num_itre):
    r = make_gaussian(x, *w) - y
    cost.append(sum(r ** 2))
    J = np.vstack((da(x, *w), db(x, *w)))
    dw = np.linalg.inv(J @ J.T + lr * np.identity(2)) @ J @ r 
    print(lr)
    new_w = w - dw
    new_r = make_gaussian(x, *new_w) - y
    new_err = sum(new_r ** 2)
    if new_err > cost[-1]:
        lr *= 100
    else:
        lr /= 100
        w = new_w
    print("Sum Squared Error:", cost[-1])
    plot_ps(x, y, w, cost)
    if np.linalg.norm(dw) <= tol:
        break

## Varying Learning Rate

In [ ]:
for learning_rate in [0.001, 0.003, 0.01]:
    cost = []
    w = np.array([10, 13,])
    for iteration in range(num_itre):
        r = make_gaussian(x, *w) - y
        cost.append(sum(r ** 2))
        J = np.vstack((da(x, *w), db(x, *w)))
        dw = np.linalg.inv(J @ J.T + learning_rate * np.identity(2)) @ J @ r 
        new_w = w - dw
        new_r = make_gaussian(x, *new_w) - y
        new_err = sum(new_r ** 2)
        if new_err > cost[-1]:
            learning_rate *= 10
        else:
            learning_rate /= 10
            w = new_w
        if np.linalg.norm(dw) <= tol:
            break
    print("Sum Squared Error:", cost[-1])
    plot_ps(x, y, w, cost)

**2. Linear least square**: You are given a bin file from the Kitti raw sequence. Estimate the ground plane from the given bin file. After estimating the ground plane, visualize this in open3d by drawing 200-300 points on the ground with a different color on top of the plot obtained from the LiDAR scan. Use RANSAC to estimate the ground plane. Will this work without RANSAC? Why or Why not? Write down the equation of the ground plane obtained and also mention the parameters used for doing RANSAC  **\[6 points\]**

Expected result is displayed here:

![output](./data/groundplane.png)

In [3]:
def read_bin_file(file_name):
    '''
    Read the bin file
    '''
    points = np.fromfile(file_name, dtype=np.float32).reshape(-1, 4)
    points = points[:,:3]                # exclude reflectance values, becomes [X Y Z]
    points = points[1::5,:]              # remove every 5th point for display speed (optional)
    return points

In [4]:
filename = "./data/000013.bin"
points = read_bin_file(filename)
print(points)
print(points.shape)
# Function used to visualize point clouds, takes a list of 3 x N numpy array as input and plots
util.visualize_pointclouds([points.T])


[[77.28644   16.049189   2.8684492]
 [77.64033   17.796824   2.8922935]
 [76.04113   18.573856   2.8447719]
 ...
 [ 3.8057039 -1.5779376 -1.7775955]
 [ 3.8148482 -1.520616  -1.77264  ]
 [ 3.829741  -1.4582629 -1.7686896]]
(24556, 3)


## Initilization

In [5]:
Points = points
Max_itr = 100
dist_threshold = 1


## Implementation

In [6]:
def ransac_algorithm():
    
    ## global variale 
    global Points, Max_itr, dist_threshold
    ground_points = []
    Final_Ground_points = []
    w = []
#     print(Final_Ground_points)
    while Max_itr:
        random.seed()
        
        inliers = []
        inliers.append(random.randint(0,len(Points)-1))
        inliers.append(random.randint(0,len(Points)-1))
        inliers.append(random.randint(0,len(Points)-1))

        # x1,y1,z1 | x2,y2,z2 | x3,y3,z3 |
        x1,y1,z1 = Points[inliers[0]]
        x2,y2,z2 = Points[inliers[1]]
        x3,y3,z3 = Points[inliers[2]]
        
        # Value of Constants for inlier plane
        a = (y2 - y1)*(z3 - z1) - (z2 - z1)*(y3 - y1)
        b = (z2 - z1)*(x3 - x1) - (x2 - x1)*(z3 - z1)
        c = (x2 - x1)*(y3 - y1) - (y2 - y1)*(x3 - x1)
        d = -(a*x1 + b*y1 + c*z1)
        
        #plane length
        Plane = max(0.1, math.sqrt(a*a+b*b+c*c))
        
        # interating in points
        for index in range(len(Points)):
            if index in inliers:
                continue
            x4,y4,z4 = Points[index]
            
            # distance calculation
            distance = math.fabs(a*x4 + b*y4 + c*z4 + d) / Plane
            
            if distance < dist_threshold:
                inliers.append(index)
            
            if len(inliers) > len(ground_points):
                ground_points.clear()
                ground_points = inliers
                w=[a,b,c,d]
        
#         for index in ground_points:
# #             print(Points.shape)
#             Final_Ground_points.append(np.array(Points[index]))
        
        
        return w


            
            
        

In [10]:
w = ransac_algorithm()

print(w)
x_min = np.amin(points,0)
x_max = np.amax(points,0)
# print(x_min,x_max)
# y_min = np.amin(points,1)
# y_max = np.amax(points,1)
# print(np.linspace(x_min,x_max,200))
pts = 50
z_points = np.linspace(x_min[2],x_max[2],pts)
y_points = np.linspace(x_min[1],x_max[1],pts)
# print(z_points)
# print(y_points)

# z1_points = np.array(points[:, [2]])
# y1_points = np.array(points[:, [1]])
# print(z1_points)
# print(y1_points)
# print(points)
# print(z_points)
# print(y_points)pcd = 
x_points = []
pcd = [np.array([0,0,0])]
for z in z_points:
#     print(z)
    x = np.array(-w[3]-w[2]*z-w[1]*y_points) / w[0]
    zz = np.full(
      shape=pts,
      fill_value=z,
      dtype=np.int
    )
#     print(zz,x,y_points)
    p = np.c_[x,y_points,zz]
#     print(p,pcd)
    pcd = np.append(pcd,p,0)
#     pcd = np.concatenate(np.array(pcd),np.array(p),0)
        
    
# print(z_points)
# pcd = np.c_[x_points,y_points,z_points]

# print(pcd)
# print(points)

util.visualize_pointclouds([np.array(points.T),np.array(pcd.T)])

[-1.425916, -3.444885, 121.65985, 205.07906]


<ipython-input-10-c97c0bf16af1>:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int


## Equation of Ground plane

In [11]:
print(w[0],"x + ",w[1],"y + ",w[2],"z + ",w[3]," = 0")

-1.425916 x +  -3.444885 y +  121.65985 z +  205.07906  = 0


## Q) Will this work without RANSAC?
## Yes, it can work without using Ransac using ML with methods like linear regression and Nearest neighbour algorithms, As in there also we need to estimate the parameters given input values and need to fit the equation for that. For our case we have more number of points corresponding to ground so linear regression will fit equation of plane nearly to ground.

# Ransac parameters 

In [12]:
print("MAX ITERATION = ",Max_itr)
print("Distance Threshold = ", dist_threshold)

MAX ITERATION =  100
Distance Threshold =  1
